In [1]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from joblib import dump, load
import pickle
from flask import Flask, jsonify, request
import numpy as np
from sklearn.externals import joblib
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.externals import joblib
from bs4 import BeautifulSoup
import re

In [ ]:
import flask
app = Flask(__name__)

In [ ]:
''' add new features'''
def add_feature(test_copy):
    test_copy['timestamp']=pd.to_datetime(test_copy['timestamp'])
    test_copy['hour']=np.uint8(test_copy['timestamp'].dt.hour)
    test_copy['day']=np.uint8(test_copy['timestamp'].dt.day)
    test_copy['month']=np.uint8(test_copy['timestamp'].dt.month)
    test_copy['weekday']=np.uint8(test_copy['timestamp'].dt.dayofweek)
    ''' change the datatype to reduce memory usage'''
    test_copy['building_id']=np.uint8(test_copy['building_id'])
    test_copy['meter']=np.uint8(test_copy['meter'])
    test_copy['site_id']=np.uint8(test_copy['site_id'])
    test_copy['square_feet']=np.uint8(test_copy['square_feet'])
    
    return test_copy

In [ ]:
def preprocessed(test_copy):
    LabelEncoder = load('LabelEncoder_primary_use.joblib')
    test_copy['primary_use'] = LabelEncoder.transform(test_copy['primary_use'])
        
    test_copy['year_built'].fillna(1969.0,inplace  = True)
    test_copy['floor_count'].fillna(3.0,inplace  = True)
    test_copy['air_temperature'].fillna(15.505707129926357,inplace  = True)
    test_copy['cloud_coverage'].fillna(1.9733458449444876,inplace  = True)
    test_copy['dew_temperature'].fillna(7.585971305989484,inplace  = True)
    test_copy['precip_depth_1_hr'].fillna(0.9182989150029545,inplace  = True)
    test_copy['sea_level_pressure'].fillna(1016.1999821450886,inplace  = True)
    test_copy['wind_direction'].fillna(173.84081711747407,inplace  = True)
    test_copy['wind_speed'].fillna(3.3861529705667808,inplace  = True)
    test_copy.drop('row_id',axis=1,inplace=True)
    test_copy.drop('timestamp',axis=1,inplace=True)
    
    return test_copy


In [ ]:
@app.route('/')
def hello_world():
    return 'Hello World!'


@app.route('/index')
def index():
    return flask.render_template('index.html')


@app.route('/predict', methods=['POST'])

In [ ]:
def predict():
        
    df_pred = pd.DataFrame()
    to_predict_list = request.form.to_dict()
    df = pd.DataFrame(to_predict_list.values(),columns=[to_predict_list.keys()])
    feature_added_df=add_feature(df)
    preprocesse_df=preprocessed(feature_added_df)
    #pred = clf.predict(df)
    stack=[]
    clf1 = joblib.load('Lasso_1.joblib')
    stack.append(clf1.predict(preprocesse_df))
    clf2 = joblib.load('Ridge_2.joblib')
    stack.append(clf2.predict(preprocesse_df))
    clf3 = joblib.load('ElasticNet_3.joblib')
    stack.append(clf3.predict(preprocesse_df))
    clf4 = joblib.load('XGBRegressor_4.joblib')
    stack.append(clf4.predict(preprocesse_df))
    clf5 = joblib.load('DecisionTreeRegressor_5.joblib')
    stack.append(clf5.predict(preprocesse_df))
    clf6 = joblib.load('LinearRegression_6.joblib')
    stack.append(clf6.predict(preprocesse_df))
    clf7 = joblib.load('Lasso_7.joblib')
    stack.append(clf7.predict(preprocesse_df))
    clf8 = joblib.load('Ridge_8.joblib')
    stack.append(clf8.predict(preprocesse_df))
    clf9 = joblib.load('ElasticNet_9.joblib')
    stack.append(clf9.predict(preprocesse_df))
    clf10 = joblib.load('XGBRegressor_10.joblib')
    stack.append(clf10.predict(preprocesse_df))
    
    stack=np.array(stack)   
    Metamodel = joblib.load('Metamodel.joblib')
    prediction=Metamodel.predict(stack)
    
    return jsonify({'prediction': prediction})
    

In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)